In [1]:
import os

In [2]:
%pwd # project working directory

'e:\\Project 4 Wine Quality MLFLOW\\Wine-quality-End-to-end-Project\\research'

In [3]:
os.chdir("../") # We go one folder back

In [4]:
%pwd

'e:\\Project 4 Wine Quality MLFLOW\\Wine-quality-End-to-end-Project'

In [49]:

# this is the code for config_entity.py
# config_entity.py contains the datatype

from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [50]:
rom mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
f

In [51]:
## this code will go to the src/mlProject/config/configuration.py
## All this code is being first written inside the jupyter notebook just for testing 
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH) :
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [52]:
import os 
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [53]:
### This code goes to the data_ingestion.py
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)


In [54]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-30 13:09:52,656: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-30 13:09:52,658: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-30 13:09:52,659: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-30 13:09:52,660: INFO: common: created directory at: artifacts]
[2024-03-30 13:09:52,660: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-30 13:09:53,833: INFO: 911009488: artifacts/data_ingestion/data.zip downloaded! with the following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "fe410515b9d87d4c19d41cfa56eeca86bdcc7116c126baaa918d4461370d5a25"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F7B0:3CA2EE:1D7252:268DD4:6607C1C8
Accept-Ranges: bytes
Date: Sa